In [1]:
import os
import numpy as np
import pandas as pd

import riskflow_jupyter

In [2]:
rundate = '2022-07-07'
if os.name=='nt':
    path = os.path.join('U:\\CVA_JSON', rundate)
    path_transform={
        '\\\\ICMJHBMVDROPPRD\\AdaptiveAnalytics\\Inbound\\MarketData':
        '\\\\ICMJHBMVDROPUAT\\AdaptiveAnalytics\\Inbound\\MarketData'}
else:
    path = os.path.join('/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON', rundate)
    path_transform={
            '//ICMJHBMVDROPPRD/AdaptiveAnalytics/Inbound/MarketData': 
            '/media/vretiel/3EFA4BCDFA4B7FDF/Media/Data/crstal/CVA_JSON'}
    
cx = riskflow_jupyter.rf.Context(
    path_transform=path_transform,
    file_transform={
        'CVAMarketData_Calibrated.dat': 'CVAMarketData_Calibrated_New.json',
        'MarketData.dat': 'MarketData.json'
    })

In [3]:
from ipywidgets import Label, HBox, VBox, ValueWidget
from riskflow_widgets import Table

In [4]:
# cx.load_json(os.path.join(path, 'InputAAJ_CrB_Russellstone_Treasury_ISDA.json'))
cx.load_json(os.path.join(path, 'InputAAJ_CrB_BNP_Paribas__Paris__ISDA.json'))


In [5]:
import ipywidgets
dir(ipywidgets)

['Accordion',
 'AppLayout',
 'Audio',
 'BoundedFloatText',
 'BoundedIntText',
 'Box',
 'Button',
 'ButtonStyle',
 'CallbackDispatcher',
 'Checkbox',
 'Color',
 'ColorPicker',
 'Combobox',
 'Controller',
 'CoreWidget',
 'DOMWidget',
 'DatePicker',
 'Datetime',
 'Dropdown',
 'FileUpload',
 'FloatLogSlider',
 'FloatProgress',
 'FloatRangeSlider',
 'FloatSlider',
 'FloatText',
 'GridBox',
 'GridspecLayout',
 'HBox',
 'HTML',
 'HTMLMath',
 'Image',
 'IntProgress',
 'IntRangeSlider',
 'IntSlider',
 'IntText',
 'Label',
 'Layout',
 'NumberFormat',
 'Output',
 'Password',
 'Play',
 'RadioButtons',
 'Select',
 'SelectMultiple',
 'SelectionRangeSlider',
 'SelectionSlider',
 'SliderStyle',
 'Style',
 'Tab',
 'Text',
 'Textarea',
 'ToggleButton',
 'ToggleButtons',
 'ToggleButtonsStyle',
 'TwoByTwoLayout',
 'VBox',
 'Valid',
 'ValueWidget',
 'Video',
 'Widget',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__jupyter_widgets_base_version__',
 '__jupyter_widgets_controls_version__',
 '__

In [6]:
factor = riskflow_jupyter.rf.utils.Factor('FXVol',('GBP','ZAR'))
vol = riskflow_jupyter.rf.riskfactors.construct_factor( factor,
    cx.current_cfg.params['Price Factors'],
    cx.current_cfg.params['Price Factor Interpolation']
)

In [7]:
import pandas as pd
sparse_matrix = {}
for k in vol.param['Surface'].array:
    sparse_matrix.setdefault(k[0], {}).setdefault(k[1], k[2])
    
df = pd.DataFrame(sparse_matrix).sort_index(0, 1)
#np.vstack() df.values
#pd.concat([df.index.to_frame(), df], axis=0)

g=np.vstack([[0.0]+df.columns.values.tolist(), 
           np.hstack([df.index.values.reshape(-1,1).tolist(),df.values])])

#df.index.to_frame()
#df.columns.to_frame()
b=to_json(g.tolist()).replace('NaN','null')

#df.reindex(sorted(df.columns), axis=1)
#df.columns

/tmp/ipykernel_347425/470012167.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  df = pd.DataFrame(sparse_matrix).sort_index(0, 1)


NameError: name 'to_json' is not defined

In [8]:
import k3d
import json


def to_json(string):
    """converts a string to json - skips the whitespace for a smaller output"""
    return json.dumps(string, separators=(',', ':'))


def load_table_from_vol(vol):
    sparse_matrix = {}
    for k in vol.param['Surface'].array:
        sparse_matrix.setdefault(k[0], {}).setdefault(k[1], k[2])

    df = pd.DataFrame(sparse_matrix).sort_index(0, 1)

    table = np.vstack([[0.0]+df.columns.values.round(5).tolist(), 
               np.hstack([df.index.values.round(5).reshape(-1,1).tolist(),
                          df.round(5).replace({np.nan:None})])]).tolist()
    
    return table


class Three(HBox):
    def __init__(self, description):
        self.description = Label(value=description)        
        self.plot = k3d.plot(
            axes=['log(moneyness)', 'expiry','vol(\%)'],
            menu_visibility=False,
            camera_rotate_speed=3.0
            )
        self.mesh = None
        self.data = Table(description='', settings=to_json({
            'width':500, 'height':300, 'contextMenu':True, 'minSpareRows':1, 'minSpareCols':1
        }))                
        
        super().__init__(
            children=[
                self.description,
                VBox(children=[self.plot, self.data])
            ]
        )

        
    @staticmethod
    def make_faces_vectorized1(Nr,Nc):

        out = np.empty((Nr-1,Nc-1,2,3),dtype=int)

        r = np.arange(Nr*Nc).reshape(Nr,Nc)

        out[:,:, 0,0] = r[:-1,:-1]
        out[:,:, 1,0] = r[:-1,1:]
        out[:,:, 0,1] = r[:-1,1:]

        out[:,:, 1,1] = r[1:,1:]
        out[:,:, :,2] = r[1:,:-1,None]

        out.shape =(-1,3)
        return out
    
    @staticmethod
    def interpolate_surface(json_list):
        moneyness = json_list[0][1:]
        e = []
        for p in json_list[1:]:
            e.extend([[m, p[0], v] for m,v in zip(moneyness, p[1:]) if v is not None])

        surface = np.array(e)
        expiry = [p[0] for p in json_list[1:]]
        return expiry, moneyness, np.array([np.interp(
            moneyness, surface[surface[:, 1] == x][:, 0], surface[surface[:, 1] == x][:, 2]) for x in expiry])

    
    @property
    def value(self):        
        return self.data.value
    
    # a setter function
    @value.setter
    def value(self, json_list):
        # self.data.{"type":"numeric","format":"0.0000"}
        
        self.data.value = to_json(json_list)
        e, moneyness, vol = Three.interpolate_surface(json_list)
        scale = 2/np.log(2)
        m = scale*np.log(moneyness)
        v = vol*100
        
        U, V = np.meshgrid(m, e)
        vertices = np.dstack([U, V, v]).reshape(-1,3)
        indices = Three.make_faces_vectorized1(*v.shape)
        if self.mesh is not None:
            self.plot-=self.mesh
        self.mesh = k3d.mesh(
            vertices.astype(np.float32), indices.astype(np.uint32), flat_shading=False, attribute=v,
            side='double', color_map=k3d.basic_color_maps.Reds, color_range=[v.min(), v.max()]
        )
        self.plot +=self.mesh
        

In [9]:
j=load_table_from_vol(vol)
j
#np.isnan(None)

/tmp/ipykernel_347425/4051841979.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  df = pd.DataFrame(sparse_matrix).sort_index(0, 1)


[[0.0,
  0.39878,
  0.43735,
  0.48505,
  0.54746,
  0.60673,
  0.63901,
  0.66332,
  0.67728,
  0.69329,
  0.72471,
  0.7297,
  0.78709,
  0.79955,
  0.80339,
  0.80654,
  0.8244,
  0.82943,
  0.8335,
  0.8493,
  0.85728,
  0.88173,
  0.89068,
  0.8996,
  0.90935,
  0.91238,
  0.92391,
  0.92753,
  0.93578,
  0.94757,
  0.94785,
  0.95619,
  0.95829,
  0.96846,
  0.97109,
  0.9788,
  0.97981,
  0.98502,
  0.98894,
  0.99428,
  1.00004,
  1.00024,
  1.00048,
  1.00106,
  1.00201,
  1.00292,
  1.00572,
  1.00588,
  1.00865,
  1.01121,
  1.01168,
  1.01509,
  1.02082,
  1.02142,
  1.02468,
  1.02942,
  1.03219,
  1.03705,
  1.04234,
  1.04516,
  1.04972,
  1.05409,
  1.05478,
  1.06257,
  1.0664,
  1.07661,
  1.07984,
  1.09408,
  1.09506,
  1.10925,
  1.11599,
  1.12938,
  1.15991,
  1.16447,
  1.18925,
  1.19613,
  1.19685,
  1.2237,
  1.22931,
  1.23505,
  1.25834,
  1.32793,
  1.33924,
  1.40235,
  1.40243,
  1.458,
  1.46925,
  1.53013,
  1.69434,
  1.86919,
  2.03357,
  2.18972],
 

In [25]:
# g = np.vstack([x[0], heights]).tolist()
t = Three(description='Test' )



In [26]:
# t.data.col_headers = ['x/y']+x[0].tolist()
t.value = j

In [27]:
t.data.settings

'{"width":500,"height":300,"contextMenu":true,"minSpareRows":1,"minSpareCols":1}'

In [28]:
t.data.value

'[[0.0,0.39878,0.43735,0.48505,0.54746,0.60673,0.63901,0.66332,0.67728,0.69329,0.72471,0.7297,0.78709,0.79955,0.80339,0.80654,0.8244,0.82943,0.8335,0.8493,0.85728,0.88173,0.89068,0.8996,0.90935,0.91238,0.92391,0.92753,0.93578,0.94757,0.94785,0.95619,0.95829,0.96846,0.97109,0.9788,0.97981,0.98502,0.98894,0.99428,1.00004,1.00024,1.00048,1.00106,1.00201,1.00292,1.00572,1.00588,1.00865,1.01121,1.01168,1.01509,1.02082,1.02142,1.02468,1.02942,1.03219,1.03705,1.04234,1.04516,1.04972,1.05409,1.05478,1.06257,1.0664,1.07661,1.07984,1.09408,1.09506,1.10925,1.11599,1.12938,1.15991,1.16447,1.18925,1.19613,1.19685,1.2237,1.22931,1.23505,1.25834,1.32793,1.33924,1.40235,1.40243,1.458,1.46925,1.53013,1.69434,1.86919,2.03357,2.18972],[0.00274,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.1673,null,0.1655,0.1625,0.16,null,null,null,null,null,0.1615,null,null,0.1665,null,null,

In [29]:
t

Three(children=(Label(value='Test'), VBox(children=(Plot(antialias=3, axes=['log(moneyness)', 'expiry', 'vol(\…

In [31]:
#pd.DataFrame(json.loads(t.data.value))
t.value = json.loads(t.data.value)

ValueError: array of sample points is empty

In [ ]:
%debug

> /home/vretiel/miniconda3/lib/python3.8/site-packages/numpy/lib/function_base.py(1570)interp()
   1568         fp = np.concatenate((fp[-1:], fp, fp[0:1]))
   1569 
-> 1570     return interp_func(x, xp, fp, left, right)
   1571 
   1572 

ipdb> u
> <__array_function__ internals>(180)interp()

ipdb> u
> /tmp/ipykernel_347425/4051841979.py(71)<listcomp>()
     69         surface = np.array(e)
     70         expiry = [p[0] for p in json_list[1:]]
---> 71         return expiry, moneyness, np.array([np.interp(
     72             moneyness, surface[surface[:, 1] == x][:, 0], surface[surface[:, 1] == x][:, 2]) for x in expiry])
     73 

ipdb> e
*** NameError: name 'e' is not defined
ipdb> u
> /tmp/ipykernel_347425/4051841979.py(71)interpolate_surface()
     69         surface = np.array(e)
     70         expiry = [p[0] for p in json_list[1:]]
---> 71         return expiry, moneyness, np.array([np.interp(
     72             moneyness, surface[surface[:, 1] == x][:, 0], surface[surface[:, 

In [24]:
t.data.value

'"[[0,0.39878,0.43735,0.48505,0.54746,0.60673,0.63901,0.66332,0.67728,0.69329,0.72471,0.7297,0.78709,0.79955,0.80339,0.80654,0.8244,0.82943,0.8335,0.8493,0.85728,0.88173,0.89068,0.8996,0.90935,0.91238,0.92391,0.92753,0.93578,0.94757,0.94785,0.95619,0.95829,0.96846,0.97109,0.9788,0.97981,0.98502,0.98894,0.99428,1.00004,1.00024,1.00048,1.00106,1.00201,1.00292,1.00572,1.00588,1.00865,1.01121,1.01168,1.01509,1.02082,1.02142,1.02468,1.02942,1.03219,1.03705,1.04234,1.04516,1.04972,1.05409,1.05478,1.06257,1.0664,1.07661,1.07984,1.09408,1.09506,1.10925,1.11599,1.12938,1.15991,1.16447,1.18925,1.19613,1.19685,1.2237,1.22931,1.23505,1.25834,1.32793,1.33924,1.40235,1.40243,1.458,1.46925,1.53013,1.69434,1.86919,2.03357,2.18972],[0.00274,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.1673,null,0.1655,0.1625,0.16,null,null,null,null,null,0.1615,null,null,0.1665,null,null,0

In [ ]:
from ipyfilechooser import FileChooser

# Create and display a FileChooser widget
fc = FileChooser('/home/vretiel/miniconda3/')
display(fc)

In [ ]:
fc.selected_filename

In [ ]:
cx

In [15]:
pp = riskflow_jupyter.PortfolioPage(cx.current_cfg)

In [16]:
pp.main_container

/home/vretiel/miniconda3/lib/python3.8/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [ ]:
rfp = riskflow_jupyter.RiskFactorsPage(cx.current_cfg)

In [ ]:
rfp.main_container


In [ ]:
rfp.tree.checked


In [ ]:
%debug

In [ ]:
import k3d